
Does going to university in a different country affect your mental health? A Japanese international university surveyed its students in 2018 and published a study the following year that was approved by several ethical and regulatory boards.

The study found that international students have a higher risk of mental health difficulties than the general population, and that social connectedness (belonging to a social group) and acculturative stress (stress associated with joining a new culture) are predictive of depression.


Exploring the `students` dataset using PostgreSQL to determine whether I would come to a similar conclusion for international students and see if the length of stay is a contributing factor.

Here is a data description of the columns that may be helpful for the analysis.

| Field Name    | Description                                      |
| ------------- | ------------------------------------------------ |
| `inter_dom`     | Types of students (international or domestic)   |
| `japanese_cate` | Japanese language proficiency                    |
| `english_cate`  | English language proficiency                     |
| `academic`      | Current academic level (undergraduate or graduate) |
| `age`           | Current age of student                           |
| `stay`          | Current length of stay in years                  |
| `todep`         | Total score of depression (PHQ-9 test)           |
| `tosc`          | Total score of social connectedness (SCS test)   |
| `toas`          | Total score of acculturative stress (ASISS test) |

In [1]:
-- Run this code to save the CSV file as students
SELECT * 
FROM 'students.csv';

,inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,english,english_cate,intimate,religion,suicide,dep,deptype,todep,depsev,tosc,apd,ahome,aph,afear,acs,aguilt,amiscell,toas,partner,friends,parents,relative,profess,phone,doctor,reli,alone,others,internet,partner_bi,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi
0,Inter,SEA,Male,Grad,24.0,4.0,5.0,Long,3.0,Average,5.0,High,NaN,Yes,No,No,No,0.0,Min,34.0,23.0,9.0,11.0,8.0,11.0,2.0,27.0,91.0,5.0,5.0,6.0,3.0,2.0,1.0,4.0,1.0,3.0,4.0,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No
1,Inter,SEA,Male,Grad,28.0,5.0,1.0,Short,4.0,High,4.0,High,NaN,No,No,No,No,2.0,Min,48.0,8.0,7.0,5.0,4.0,3.0,2.0,10.0,39.0,7.0,7.0,7.0,4.0,4.0,4.0,4.0,1.0,1.0,1.0,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No
2,Inter,SEA,Male,Grad,25.0,4.0,6.0,Long,4.0,High,4.0,High,Yes,Yes,No,No,No,2.0,Min,41.0,13.0,4.0,7.0,6.0,4.0,3.0,14.0,51.0,3.0,3.0,3.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,NaN,No,No,No,No,No,No,No,No,No,No,No
3,Inter,EA,Female,Grad,29.0,5.0,1.0,Short,2.0,Low,3.0,Average,No,No,No,No,No,3.0,Min,37.0,16.0,10.0,10.0,8.0,6.0,4.0,21.0,75.0,5.0,5.0,5.0,5.0,5.0,2.0,2.0,2.0,4.0,4.0,NaN,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No
4,Inter,EA,Female,Grad,28.0,5.0,1.0,Short,1.0,Low,3.0,Average,Yes,No,No,No,No,3.0,Min,37.0,15.0,12.0,5.0,8.0,7.0,4.0,31.0,82.0,5.0,5.0,5.0,2.0,5.0,2.0,5.0,5.0,4.0,4.0,NaN,Yes,Yes,Yes,No,Yes,No,Yes,Yes,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46,222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
282,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65,203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The following are the exploratory steps.

I start by counting all of the records in the dataset.

In [2]:
SELECT COUNT(*)
FROM 'students.csv';

,count_star()
0,286


Then, I count all records perstudent type to see how the records are categorized and scored.

In [3]:
SELECT inter_dom, COUNT(inter_dom) AS count_inter_dom, COUNT(*) AS total_records
FROM 'students.csv'
GROUP BY inter_dom;

,inter_dom,count_inter_dom,total_records
0,Inter,201,201
1,Dom,67,67
2,NaN,0,18


There are 201 records are international and 67 are domestic. However, the table also has 18 with NULL value.

Next, I will filter the data to see how it differs between the students types.

In [4]:
SELECT *
FROM 'students.csv'
WHERE inter_dom = 'Inter';

,inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,english,english_cate,intimate,religion,suicide,dep,deptype,todep,depsev,tosc,apd,ahome,aph,afear,acs,aguilt,amiscell,toas,partner,friends,parents,relative,profess,phone,doctor,reli,alone,others,internet,partner_bi,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi
0,Inter,SEA,Male,Grad,24,4,5,Long,3,Average,5,High,NaN,Yes,No,No,No,0,Min,34,23,9,11,8,11,2,27,91,5,5,6,3,2,1,4,1,3,4,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No
1,Inter,SEA,Male,Grad,28,5,1,Short,4,High,4,High,NaN,No,No,No,No,2,Min,48,8,7,5,4,3,2,10,39,7,7,7,4,4,4,4,1,1,1,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No
2,Inter,SEA,Male,Grad,25,4,6,Long,4,High,4,High,Yes,Yes,No,No,No,2,Min,41,13,4,7,6,4,3,14,51,3,3,3,1,1,2,1,1,1,1,NaN,No,No,No,No,No,No,No,No,No,No,No
3,Inter,EA,Female,Grad,29,5,1,Short,2,Low,3,Average,No,No,No,No,No,3,Min,37,16,10,10,8,6,4,21,75,5,5,5,5,5,2,2,2,4,4,NaN,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No
4,Inter,EA,Female,Grad,28,5,1,Short,1,Low,3,Average,Yes,No,No,No,No,3,Min,37,15,12,5,8,7,4,31,82,5,5,5,2,5,2,5,5,4,4,NaN,Yes,Yes,Yes,No,Yes,No,Yes,Yes,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,Inter,SEA,Male,Under,21,3,3,Medium,4,High,4,High,Yes,Yes,No,No,No,6,Mild,37,23,8,11,8,6,4,22,82,3,2,2,2,4,2,4,2,6,4,4.0,No,No,No,No,No,No,No,No,Yes,No,No
197,Inter,SEA,Female,Under,20,2,1,Short,2,Low,4,High,Yes,No,No,No,No,7,Mild,16,29,18,17,17,12,8,44,145,7,1,3,3,4,4,4,4,7,1,4.0,Yes,No,No,No,No,No,No,No,Yes,No,No
198,Inter,SEA,Female,Under,21,3,3,Medium,2,Low,5,High,Yes,No,No,Yes,Major,16,ModSev,25,24,11,17,4,11,6,37,110,5,7,3,1,6,1,6,1,4,1,3.0,Yes,Yes,No,No,Yes,No,Yes,No,No,No,No
199,Inter,SEA,Female,Under,18,1,1,Short,1,Low,4,High,No,No,No,No,No,8,Mild,38,11,12,10,4,7,4,20,68,5,5,4,3,3,3,3,3,5,5,5.0,Yes,Yes,No,No,No,No,No,No,Yes,Yes,Yes


In [5]:
SELECT *
FROM 'students.csv'
WHERE inter_dom = 'Dom';

,inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,english,english_cate,intimate,religion,suicide,dep,deptype,todep,depsev,tosc,apd,ahome,aph,afear,acs,aguilt,amiscell,toas,partner,friends,parents,relative,profess,phone,doctor,reli,alone,others,internet,partner_bi,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi
0,Dom,JAP,Female,Grad,27,5,2,Medium,3,Average,3,Average,Yes,Yes,No,Yes,Major,12,Mod,47,16,11,5,8,7,3,31,81,7,3,7,1,6,6,1,5,4,1,NaN,Yes,No,Yes,No,Yes,Yes,No,Yes,No,No,No
1,Dom,JAP,Female,Under,18,1,1,Short,5,High,3,Average,No,No,No,No,No,9,Mild,48,9,4,5,4,3,2,10,37,4,4,4,4,1,1,1,1,1,1,4.0,No,No,No,No,No,No,No,No,No,No,No
2,Dom,JAP,Female,Under,21,3,3,Medium,5,High,3,Average,Yes,No,No,No,No,7,Mild,40,16,8,10,8,6,4,20,72,6,6,7,1,1,1,5,1,1,1,4.0,Yes,Yes,Yes,No,No,No,Yes,No,No,No,No
3,Dom,JAP,Male,Under,20,2,3,Medium,5,High,1,Low,No,No,No,No,No,3,Min,47,11,4,5,4,5,2,12,43,1,5,5,3,1,1,3,1,1,1,3.0,No,Yes,Yes,No,No,No,No,No,No,No,No
4,Dom,JAP,Female,Under,21,3,3,Medium,5,High,1,Low,No,No,Yes,Yes,Other,10,Mod,48,8,4,5,4,3,2,10,36,7,5,7,1,1,1,1,1,1,1,1.0,Yes,Yes,Yes,No,No,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Dom,JAP,Female,Under,21,3,4,Long,5,High,4,High,No,Yes,No,No,No,8,Mild,27,16,9,10,8,7,4,20,74,1,7,5,1,3,3,3,1,1,1,6.0,No,Yes,Yes,No,No,No,No,No,No,No,Yes
63,Dom,JAP,Female,Under,22,3,3,Medium,3,Average,4,High,Yes,Yes,No,No,No,2,Min,48,8,10,5,4,3,4,16,50,7,7,7,7,2,2,2,2,2,1,3.0,Yes,Yes,Yes,Yes,No,No,No,No,No,No,No
64,Dom,JAP,Female,Under,19,2,1,Short,5,High,3,Average,No,No,No,No,No,9,Mild,47,8,7,5,5,3,2,13,43,5,7,7,6,7,7,7,1,1,1,2.0,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No
65,Dom,JAP,Male,Under,19,2,1,Short,5,High,3,Average,No,No,No,No,No,1,Min,43,8,12,5,4,3,2,10,44,7,5,7,5,5,5,5,4,4,4,2.0,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,No


In [6]:
SELECT *
FROM 'students.csv'
WHERE inter_dom IS NULL;

,inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,english,english_cate,intimate,religion,suicide,dep,deptype,todep,depsev,tosc,apd,ahome,aph,afear,acs,aguilt,amiscell,toas,partner,friends,parents,relative,profess,phone,doctor,reli,alone,others,internet,partner_bi,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96,42,NaN,65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145,128,137,66,61,30,46,19,65,21,45
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,172,54,NaN,107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123,140,131,202,207,238,222,249,203,247,223
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,172,NaN,73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145,128,137,66,61,30,46,19,65,21,45
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123,140,131,202,207,238,222,249,203,247,223
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145,123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128,140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137,131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As we can see with international student type, we have students are in both undergraduate and graduate academic levels with age range is from 17 to over 30. On the other hand, students who do not study aboard are all undergraduate students, and all of them are under 30.

With NULL records, there are clearly no valuable information.

I am going to find the summary statistics of the diagnostic tests for all students.

In [2]:
SELECT 'Average', ROUND(AVG(todep),2) AS 'Score of depression (PHQ-9 test)', ROUND(AVG(tosc), 2) AS 'Score of social connectedness (SCS test)', ROUND(AVG(toas), 2) AS 'Score of acculturative stress (ASISS test)'
FROM 'students.csv'
UNION
SELECT 'Standard deviation', ROUND(STDDEV(todep), 2) AS 'Score of depression (PHQ-9 test)', ROUND(STDDEV(tosc), 2) AS 'Score of social connectedness (SCS test)', ROUND(STDDEV(toas),2) AS 'Score of acculturative stress (ASISS test)'
FROM 'students.csv'
UNION
SELECT 'Max', MAX(todep) AS 'Score of depression (PHQ-9 test)', MAX(tosc) AS 'Score of social connectedness (SCS test)', MAX(toas) AS 'Score of acculturative stress (ASISS test)'
FROM 'students.csv'
UNION
SELECT 'Min', MIN(todep) AS 'Score of depression (PHQ-9 test)', MIN(tosc) AS 'Score of social connectedness (SCS test)', MIN(toas) AS 'Score of acculturative stress (ASISS test)'
FROM 'students.csv';

,'Average',Score of depression (PHQ-9 test),Score of social connectedness (SCS test),Score of acculturative stress (ASISS test)
0,Max,25.00,48.00,145.00
1,Standard deviation,4.95,9.23,22.64
2,Average,8.19,37.47,72.38
3,Min,0.00,8.00,36.00


I am going to repeat the summary statistics steps but this time only applies to international students.

In [1]:
SELECT 'Average', ROUND(AVG(todep),2) AS 'Score of depression (PHQ-9 test)', ROUND(AVG(tosc), 2) AS 'Score of social connectedness (SCS test)', ROUND(AVG(toas), 2) AS 'Score of acculturative stress (ASISS test)'
FROM 'students.csv'
WHERE inter_dom = 'Inter'
UNION
SELECT 'Standard deviation', ROUND(STDDEV(todep), 2) AS 'Score of depression (PHQ-9 test)', ROUND(STDDEV(tosc), 2) AS 'Score of social connectedness (SCS test)', ROUND(STDDEV(toas),2) AS 'Score of acculturative stress (ASISS test)'
FROM 'students.csv'
WHERE inter_dom = 'Inter'
UNION
SELECT 'Max', MAX(todep) AS 'Score of depression (PHQ-9 test)', MAX(tosc) AS 'Score of social connectedness (SCS test)', MAX(toas) AS 'Score of acculturative stress (ASISS test)'
FROM 'students.csv'
WHERE inter_dom = 'Inter'
UNION
SELECT 'Min', MIN(todep) AS 'Score of depression (PHQ-9 test)', MIN(tosc) AS 'Score of social connectedness (SCS test)', MIN(toas) AS 'Score of acculturative stress (ASISS test)'
FROM 'students.csv'
WHERE inter_dom = 'Inter';

,'Average',Score of depression (PHQ-9 test),Score of social connectedness (SCS test),Score of acculturative stress (ASISS test)
0,Max,25.00,48.00,145.00
1,Average,8.04,37.42,75.56
2,Standard deviation,4.90,9.13,22.56
3,Min,0.00,11.00,36.00


International students seem to have the same diagnostic scores as total records, with the average score of depression on the PHQ-9 test at 8.04 compared to 8.19 for all records. The average score of social connectedness on the SCS test is 37.42 compared to 37.47, and the average score of acculturative stress in ASISS test is 75.56 compared to  72.38. However, those information tests do not give accurate information for us to come up with a conclusion because the number of international students is the vast majority in the dataset, 75%.

To see whether the length of stay impacts the average diagnostic scores for international students, this is the final step of the analysis to answer the question if international students have a higher risk of mental health difficulties than the general population.

In [9]:
SELECT stay, ROUND(AVG(todep),2) AS average_phq, ROUND(AVG(tosc), 2) AS average_scs, ROUND(AVG(toas), 2) AS average_as
FROM 'students.csv'
WHERE inter_dom = 'Inter'
GROUP BY stay
ORDER BY stay DESC;

,stay,average_phq,average_scs,average_as
0,10,13.00,32.00,50.00
1,8,10.00,44.00,65.00
2,7,4.00,48.00,45.00
3,6,6.00,38.00,58.67
4,5,0.00,34.00,91.00
5,4,8.57,33.93,87.71
6,3,9.09,37.13,78.00
7,2,8.28,37.08,77.67
8,1,7.48,38.11,72.80


In conclusion, the longer international students stay away from home, the higher chance they will struggle with mental health problems. The average score of depression increased from 7.48 in the first year to 13 in the 10th year, which increased by 73.8%. 

To answer whether going to university in a different country affects your mental health. The answer is YES. International students who live far away from their home countries tend to face a higher risk of mental health difficulties than the general population. 

Here are some resources that international students may find helpful in order to help them overcome their mental health struggles:

1. "How to look after your mental health as an international student" https://studee.com/guides/how-to-look-after-your-mental-health-as-an-international-student/
2. "6 Mental Health Tips for International Students" https://www.usnews.com/education/best-colleges/articles/2019-04-08/how-to-look-after-mental-health-as-an-international-studentin-the-us
3. "Mental Health Advice for International Students" https://shorelight.com/student-stories/mental-health-advice-for-international-students/

